# 数值梯度进阶与非自洽 GGA 解析梯度

在这一节中，我们会了解非自洽 GGA 的解析梯度求法；这将是了解 XYG3 解析梯度求法的最初一步．同时，我们会在这一节中接着 [RHF 数值梯度](hf_nuc_grad.ipynb#数值梯度)，对数值梯度的精细的项进行分析，以确定一阶 CP-KS 方程与能量分项的数值梯度．由于笔记的内容设计，这次我们会详细地讨论 CP-KS 方程的使用与一阶梯度的简化，包括一般方法的解析梯度中不需解 U 矩阵的原因，以及在非自洽 GGA 解析梯度方法中所使用的 Z-Vector 技巧．

In [1]:
from pyscf import gto, scf, grad, data, lib, dft
import numpy as np
import pyscf.grad.rks
from pyscf.dft import numint
np.set_printoptions(8, linewidth=150, suppress=True)
EINOPT = ["greedy", 1024**3 * 2 / 8]

## 准备工作

In [2]:
mol = gto.Mole()
mol.atom = """
O  0.0  0.0  0.0
O  0.0  0.0  1.5
H  1.0  0.0  0.0
H  0.0  1.0  1.5
"""
mol.basis = "6-31G"
mol.verbose = -2
mol.build()

### 自洽场部分

尽管非自洽 GGA 能量计算的最重要一步是使用非自洽 GGA 泛函并代入自洽场 GGA 密度；但一般来说，最常用的项都在来自于自洽场泛函．如果一些项会因泛函的形式不同而不同，那么我们会对自洽场泛函加入标签 `s`，而对非自洽泛函加入标签 `n`．

In [3]:
%%time
scf_eng = dft.RKS(mol)
scf_eng.xc = "b3lypg"  # compare that to gaussian
scf_eng.grids.atom_grid = (99, 590)
scf_eng.grids.build()
scf_eng.conv_tol = 1e-13
energy_RKS = scf_eng.kernel()

CPU times: user 37 s, sys: 8.22 s, total: 45.3 s
Wall time: 2.85 s


In [4]:
%%time
# Kernel insensitive
nao = mol.nao
nmo = scf_eng.mo_energy.shape[0]
nelec = mol.nelectron
nocc = mol.nelec[0]
nvir = nmo - nocc
C = scf_eng.mo_coeff
Co = C[:, :nocc]
Cv = C[:, nocc:]
e = scf_eng.mo_energy
eo = e[:nocc]
ev = e[nocc:]
mo_occ = scf_eng.mo_occ
D = scf_eng.make_rdm1()

CPU times: user 3.25 ms, sys: 0 ns, total: 3.25 ms
Wall time: 235 µs


In [5]:
c_s_x = scf_eng._numint.hybrid_coeff(scf_eng.xc)

### 非自洽部分

非自洽 GGA 的具体计算过程大致描述如下：首先我们计算一个标准的“自洽场泛函” $E_\mathrm{xc}^\mathrm{s} [\rho] = \int f^\mathrm{xc, s}(\rho) \rho(\boldsymbol{r}) \, \mathrm{d} \boldsymbol{r}$ 的 SCF 收敛过程；在这篇文档中，我们会使用 B3LYP 作为自洽场泛函．通过自洽场泛函，我们可以得到轨道系数 $C_{\mu p}$ (或以此生成的电子态密度 $D_{\mu \nu} = 2 C_{\mu i} C_{\nu i}$)．

这套轨道系数 (密度) 将会代入“非自洽泛函”中，计算单点能 $E_\mathrm{xc}^\mathrm{n} [\rho] = \int f^\mathrm{xc, n}(\rho) \rho(\boldsymbol{r}) \, \mathrm{d} \boldsymbol{r}$．最后获得的单点能即是我们所期望的非自洽 GGA 能量．在这篇文档中，我们会使用 X3LYP 作为非自洽泛函．

B3LYP 与 X3LYP 均使用与 Gaussian 软件中相同的版本进行计算．

在 PySCF 中，其计算过程相对简单，不需要动底层代码；我们曾经也依照此方法计算 [XYG3 能量](xyg3_energy.ipynb#XYG3-能量)．这里就是一个回顾了．

In [6]:
%%time
nscf_eng = dft.RKS(mol)
nscf_eng.xc = "x3lypg"
nscf_eng.grids.atom_grid = (99, 590)
nscf_eng.grids.build()
energy_nRKS = nscf_eng.energy_tot(D)

CPU times: user 3.56 s, sys: 518 ms, total: 4.08 s
Wall time: 255 ms


除了交换积分的系数 (X3LYP 中为 $c_\mathrm{x}^\mathrm{n} = 0.218$)，我们以后会使用或多次使用非自洽泛函在当前自洽场密度下的 Fock 矩阵，以及其导出的能量加权密度矩阵．这些定义放在下述代码块中．

In [7]:
%%time
# Kernel sensitive
c_n_x = scf_eng._numint.hybrid_coeff(nscf_eng.xc)
F_n = C.T @ nscf_eng.get_fock(dm=D) @ C
F_n_occ = Co.T @ nscf_eng.get_fock(dm=D) @ Co
De_n = np.einsum("ui, ij, vj -> uv", Co, F_n_occ, Co) * 2

CPU times: user 6.56 s, sys: 991 ms, total: 7.55 s
Wall time: 472 ms


### 积分

In [8]:
%%time
natm = mol.natm

# grad-contrib
int1e_ipovlp = mol.intor('int1e_ipovlp')
int1e_ipkin = mol.intor("int1e_ipkin")
int1e_ipnuc = mol.intor("int1e_ipnuc")
int2e_ip1 = mol.intor("int2e_ip1")

# hess-contrib
int1e_ipipkin = mol.intor("int1e_ipipkin")
int1e_ipkinip = mol.intor("int1e_ipkinip")
int1e_ipipnuc = mol.intor("int1e_ipipnuc")
int1e_ipnucip = mol.intor("int1e_ipnucip")
int2e_ipip1 = mol.intor("int2e_ipip1")
int2e_ipvip1 = mol.intor("int2e_ipvip1")
int2e_ip1ip2 = mol.intor("int2e_ip1ip2")
int1e_ipipovlp = mol.intor("int1e_ipipovlp")
int1e_ipovlpip = mol.intor("int1e_ipovlpip")

def mol_slice(atm_id):
    _, _, p0, p1 = mol.aoslice_by_atom()[atm_id]
    return slice(p0, p1)

CPU times: user 2.39 s, sys: 205 ms, total: 2.59 s
Wall time: 163 ms


### DFT 格点

自洽泛函与非自洽泛函的差别在 DFT 格点上会有差异，但这并不影响 $\phi_g, w_g$ 等原子轨道和格点权重的信息．因此，下面代码块中的结果是与使用泛函形式无关的：

In [9]:
%%time
grid_ao, grid_mask, grid_weight, grid_coord = next(scf_eng._numint.block_loop(mol, scf_eng.grids, nao, 3, 2000))
ngrid = grid_ao.shape[1]

grid_ao_0  = grid_ao[0]
grid_ao_1  = grid_ao[1:4]
grid_ao_2T = grid_ao[4:10]

XX, XY, XZ, YY, YZ, ZZ = range(4, 10)
XXX, XXY, XXZ, XYY, XYZ, XZZ, YYY, YYZ, YZZ, ZZZ = range(10, 20)

grid_ao_2 = np.array([
    [grid_ao[XX], grid_ao[XY], grid_ao[XZ]],
    [grid_ao[XY], grid_ao[YY], grid_ao[YZ]],
    [grid_ao[XZ], grid_ao[YZ], grid_ao[ZZ]],
])
grid_ao_3T = np.array([
    [grid_ao[XXX], grid_ao[XXY], grid_ao[XXZ], grid_ao[XYY], grid_ao[XYZ], grid_ao[XZZ]],
    [grid_ao[XXY], grid_ao[XYY], grid_ao[XYZ], grid_ao[YYY], grid_ao[YYZ], grid_ao[YZZ]],
    [grid_ao[XXZ], grid_ao[XYZ], grid_ao[XZZ], grid_ao[YYZ], grid_ao[YZZ], grid_ao[ZZZ]],
])

grid_rho = np.einsum("uv, tgu, gv -> tg", D, grid_ao[0:4], grid_ao_0, optimize=True)
grid_rho[1:] *= 2
grid_rho_0 = grid_rho[0]
grid_rho_1 = grid_rho[1:4]

grid_A_rho_1 = np.zeros((natm, 3, ngrid))
grid_A_rho_2 = np.zeros((natm, 3, 3, ngrid))
for A in range(natm):
    sA = mol_slice(A)
    grid_A_rho_1[A]  = 2 * np.einsum("tgk ,  gl, kl -> tg ", grid_ao_1[:, :, sA],    grid_ao_0, D[sA], optimize=True)
    grid_A_rho_2[A]  = 2 * np.einsum("trgk,  gl, kl -> trg", grid_ao_2[:, :, :, sA], grid_ao_0, D[sA], optimize=True)
    grid_A_rho_2[A] += 2 * np.einsum("tgk , rgl, kl -> trg", grid_ao_1[:, :, sA],    grid_ao_1, D[sA], optimize=True)

CPU times: user 9.41 s, sys: 2.14 s, total: 11.6 s
Wall time: 768 ms


但下面与泛函核有关的格点则需要区别对待：

In [10]:
%%time
# SCF
grid_svxc, grid_sfxc = scf_eng._numint.eval_xc(scf_eng.xc, grid_rho, deriv=2)[1:3]
grid_sfr, grid_sfg = grid_svxc[0:2]
grid_sfrr, grid_sfrg, grid_sfgg = grid_sfxc[0:3]
grid_sfr  *= grid_weight
grid_sfg  *= grid_weight
grid_sfrr *= grid_weight
grid_sfrg *= grid_weight
grid_sfgg *= grid_weight
# NSCF
grid_nvxc, grid_nfxc = nscf_eng._numint.eval_xc(nscf_eng.xc, grid_rho, deriv=2)[1:3]
grid_nfr, grid_nfg = grid_nvxc[0:2]
grid_nfrr, grid_nfrg, grid_nfgg = grid_nfxc[0:3]
grid_nfr  *= grid_weight
grid_nfg  *= grid_weight
grid_nfrr *= grid_weight
grid_nfrg *= grid_weight
grid_nfgg *= grid_weight

CPU times: user 3.24 s, sys: 26.3 ms, total: 3.27 s
Wall time: 327 ms


### 非自洽 GGA 的数值梯度

我们简单地回顾一下 [数值梯度](hf_nuc_grad.ipynb#数值梯度) 的代码即可完成非自洽数值梯度的计算．需要注意的仅仅是能量的生成方式不能再通过简单的调用 PySCF 高层函数而已．

执行下述代码块的耗时会很长，因为 DFT 格点求和不管在 PySCF，还是手动地用 NumPy 的计算耗时都非常长．如果不打算等下述的代码计算过程，可以直接参考输出结果．

In [11]:
%%time
def my_nscf_energy(mol, sxc, nxc):
    smf = dft.RKS(mol)
    smf.xc = sxc
    smf.grids.atom_grid = (99, 590)
    smf.grids.build()
    smf.conv_tol = 1e-13
    smf.kernel()
    d = smf.make_rdm1()
    nmf = dft.RKS(mol)
    nmf.xc = nxc
    nmf.grids.atom_grid = (99, 590)
    nmf.grids.build()
    energy = nmf.energy_tot(d)
    return energy
    
def my_nscf_grad_numerical(mol, interval = 0.0001):
    # unit of interval is Angstrom
    natm = mol.natm
    grad_numerical = np.zeros((natm, 3))
    for atm_id in range(natm):
        for t in range(3):
            mol_change = mol.copy()
            mol_change.verbose = 0
            mol_change_coords = mol_change.atom_coords() * lib.param.BOHR
            mol_change_coords[atm_id, t] -= interval
            mol_change.set_geom_(mol_change_coords)
            e0 = my_nscf_energy(mol_change, "b3lypg", "x3lypg")
            mol_change_coords[atm_id, t] += 2 * interval
            mol_change.set_geom_(mol_change_coords)
            e1 = my_nscf_energy(mol_change, "b3lypg", "x3lypg")
            grad_numerical[atm_id, t] = (e1 - e0) / (2 * interval / lib.param.BOHR)
    return grad_numerical

grad_nscf_numerical = my_nscf_grad_numerical(mol)

CPU times: user 16min 47s, sys: 2min 58s, total: 19min 46s
Wall time: 1min 14s


In [12]:
grad_nscf_numerical

array([[-0.02778846,  0.0107175 ,  0.0161896 ],
       [ 0.0107175 , -0.02778865, -0.01618961],
       [ 0.01057407,  0.00649687,  0.02582158],
       [ 0.00649686,  0.01057408, -0.02582156]])

在 [RHF 数值梯度](hf_nuc_grad.ipynb#数值梯度) 笔记中，我们能发现数值梯度相对于解析梯度的精度一般能达到 $10^{-7} \mathrm{a.u.}$ 的精度；对于 GGA，或者非自洽 GGA 而言，数值梯度与解析梯度相比的精度似乎不能达到这个数量级，原因可能是我们并没有在解析梯度中考虑格点移动所产生的梯度，因此精度只能达到 $10^{-6} \mathrm{a.u.}$．

## 非自洽 GGA 一阶核坐标梯度

### 非 U 矩阵部分推导与实现

我们较为仔细地推导一次非自洽 GGA 的一阶核坐标梯度公式．

首先，能量项可以被拆分为

\begin{equation}
E^\mathrm{n}_\mathrm{tot} = E_\mathrm{nuc} + 2 h_{ii} + 2 (ii|jj) - c_\mathrm{x}^\mathrm{n} (ij|ij) + w_g f^\mathrm{n}_g \rho_g
\end{equation}

我们首先不考虑原子核间相互作用的贡献；这部分的贡献只与分子构型有关，但不与能量导出方法有关．

随后，我们很容易地想到要拆分偏导数为 Skeleton 部分与 U 矩阵部分．前一部分的贡献一方面较为简单，另一方面非自洽与自洽部分的区别从代码上讲并不大，因此我们就先考虑 Skeleton 部分的贡献．

\begin{align}
\partial_{A_t} E_\mathrm{elec}^\mathrm{n} &= D_{\mu \nu} \partial_{A_t} h_{\mu \nu} + \frac{1}{2} D_{\mu \nu} D_{\kappa \lambda} \partial_{A_t} (\mu \nu | \kappa \lambda) - \frac{1}{4} c_\mathrm{x}^\mathrm{n} D_{\mu \nu} D_{\kappa \lambda} \partial_{A_t} (\mu \kappa | \nu \lambda) \\
&\quad - w_g f^{\mathrm{n}, \rho}_g \rho_{Atg} - 2 w_g f^{\mathrm{n}, \gamma}_g \rho_{rg} \rho_{Atrg}
\end{align}

关于这些代码，我们可以很方便地从以前的 [GGA 梯度笔记](dft_nuc_hess.ipynb#GGA-一阶核坐标梯度) 中获得；但需要注意到，上面的代码中不包含一般的一阶梯度中所具有的 $- 2 S_{ii}^{A_t} \varepsilon_i$ 的贡献．这部分的贡献是从 U 矩阵相关部分导出的，即使它从公式上和程序上都不需要直接地计算 U 矩阵；同时，我们指出在非自洽泛函 GGA 梯度中，U 矩阵相关部分的贡献结果并不是 $- 2 S_{ii}^{A_t} \varepsilon_i$．

In [13]:
%%time
grad_skeleton = np.zeros((mol.natm, 3))
integrals_skeleton  = - int1e_ipkin - int1e_ipnuc \
                      - np.einsum("tuvkl, kl -> tuv", int2e_ip1, D) \
                      + c_n_x * 0.5 * np.einsum("tukvl, kl -> tuv", int2e_ip1, D)
for A in range(mol.natm):
    sA = mol_slice(A)
    grad_skeleton[A, :] += np.einsum("tuv, uv -> t", integrals_skeleton[:, sA], D[sA]) * 2
    with mol.with_rinv_as_nucleus(A):
        grad_skeleton[A, :] -= np.einsum("tuv, uv -> t", mol.intor("int1e_iprinv") * mol.atom_charge(A), D) * 2

grad_skeleton -=     np.einsum("g, Atg      -> At", grid_nfr, grid_A_rho_1)
grad_skeleton -= 2 * np.einsum("g, Atrg, rg -> At", grid_nfg, grid_A_rho_2, grid_rho_1)

CPU times: user 296 ms, sys: 2.41 ms, total: 298 ms
Wall time: 18.7 ms


In [14]:
grad_nuc = np.zeros((mol.natm, 3))
for A in range(mol.natm):
    for M in range(mol.natm):
        if A == M: continue;
        vec = mol.atom_coord(A) - mol.atom_coord(M)
        grad_nuc[A, :] -= mol.atom_charge(A) * mol.atom_charge(M) * vec / np.linalg.norm(vec) ** 3

### U 矩阵关联项推导与实现

借用 U 矩阵与系数矩阵之间的关系，我们应当可以很快地推导出 U 矩阵的关联项：

\begin{equation}
\partial_{A_t}^\mathrm{U} E_\mathrm{elec}^\mathrm{n} = 4 U_{pi}^{A_t} C_{\mu p} C_{\mu i} \left( h_{\mu \nu} + J_{\mu \nu} [\mathbf{D}] - \frac{1}{2} c_\mathrm{x}^\mathrm{s} K_{\mu \nu} [\mathbf{D}] + f_g^{\mathrm{n}, \rho} \phi_{g \mu} \phi_{g \nu} + 2 f_g^{\mathrm{n}, \gamma} \rho_r (\phi_{rg \mu} \phi_{g \nu}) \right)
\end{equation}

我们指出，上式的大括号中被合并的项事实上就是非自洽泛函的 Fock 矩阵在原子轨道坐标下的表示矩阵 $F_{\mu \nu}^\mathrm{n}$；因此上式很快地写为：

\begin{equation}
\partial_{A_t}^\mathrm{U} E_\mathrm{elec}^\mathrm{n} = 4 U_{pi}^{A_t} F_{pi}^\mathrm{n}
\end{equation}

如果不是非自洽过程，上式可以很轻松地被简化：因为对于自洽场方法，分子轨道下 Fock 矩阵即是对角矩阵，因此

\begin{equation}
4 U_{pi}^{A_t} F_{ai}^\mathrm{s} = 4 U_{ii} ^{A_t} \varepsilon_i = - 2 S_{ii}^{A_t} \varepsilon_i
\end{equation}

这对非自洽过程不成立，因为一般地，$F_{pq} \neq 0$，不论 $p = q$ 或 $p \neq q$．因此，这就要求我们求出 $U_{pi}^{A_t}$；这个求解过程在 Hessian 的推导中大致叙述．需要了解的是，由于 U 矩阵的求取方式是对自洽场导出的 $F_{pq}^\mathrm{s}$ 的坐标导数为零，因此求取 U 矩阵的过程使用的始终是自洽场泛函，因此描述 U 矩阵的方式从代码上可以完全与以前的代码相同：

In [15]:
%%time
cpks_ao_hgga  = -0.5 * np.einsum("g,     Atg ,      gu, gv -> Atuv", grid_sfrr,               grid_A_rho_1,             grid_ao_0, grid_ao_0, optimize=EINOPT)
cpks_ao_hgga += -  2 * np.einsum("g,     Atg , wg, wgu, gv -> Atuv", grid_sfrg,               grid_A_rho_1, grid_rho_1, grid_ao_1, grid_ao_0, optimize=EINOPT)
cpks_ao_hgga += -      np.einsum("g, rg, Atrg,      gu, gv -> Atuv", grid_sfrg, grid_rho_1,   grid_A_rho_2,             grid_ao_0, grid_ao_0, optimize=EINOPT)
cpks_ao_hgga += -  4 * np.einsum("g, rg, Atrg, wg, wgu, gv -> Atuv", grid_sfgg, grid_rho_1,   grid_A_rho_2, grid_rho_1, grid_ao_1, grid_ao_0, optimize=EINOPT)
cpks_ao_hgga += -  2 * np.einsum("g,     Atrg,     rgu, gv -> Atuv", grid_sfg,                grid_A_rho_2,             grid_ao_1, grid_ao_0, optimize=EINOPT)

cpks_ao_hgga_mat1  = -   np.einsum("g,      tgu,  gv -> tuv", grid_sfr,             grid_ao_1, grid_ao_0, optimize=EINOPT)
cpks_ao_hgga_mat1 -= 2 * np.einsum("g, rg,  tgu, rgv -> tuv", grid_sfg, grid_rho_1, grid_ao_1, grid_ao_1, optimize=EINOPT)
cpks_ao_hgga_mat1 -= 2 * np.einsum("g, rg, trgu,  gv -> tuv", grid_sfg, grid_rho_1, grid_ao_2, grid_ao_0, optimize=EINOPT)

for A in range(natm):
    sA = mol_slice(A)
    cpks_ao_hgga[A, :, sA] += cpks_ao_hgga_mat1[:, sA]
    
cpks_ao_hgga += cpks_ao_hgga.swapaxes(2, 3)

CPU times: user 19.3 s, sys: 1.25 s, total: 20.6 s
Wall time: 1.38 s


In [16]:
%%time
grid_rho1phi1 = np.einsum("tg, tgu -> gu", grid_rho_1, grid_ao_1, optimize=True)
rdm2_inAx  =   2 * np.einsum("g,  gk,  gu -> gku", grid_sfrg, grid_ao_0,     grid_rho1phi1, optimize=EINOPT)
rdm2_inAx += rdm2_inAx.swapaxes(1, 2)
rdm2_inAx += 0.5 * np.einsum("g,  gk,  gu -> gku", grid_sfrr, grid_ao_0,     grid_ao_0,     optimize=EINOPT)
rdm2_inAx +=   8 * np.einsum("g,  gk,  gu -> gku", grid_sfgg, grid_rho1phi1, grid_rho1phi1, optimize=EINOPT)
rdm2_inAx +=   4 * np.einsum("g, rgk, rgu -> gku", grid_sfg,  grid_ao_1,     grid_ao_1,     optimize=EINOPT)
rdm2_inAx = np.einsum("gku, gl, gv -> kluv", rdm2_inAx, grid_ao_0, grid_ao_0, optimize=EINOPT)

CPU times: user 29 s, sys: 835 ms, total: 29.9 s
Wall time: 2.56 s


In [17]:
def Axao(x, C):
    shape = list(x.shape)
    shape[-1] = nao; shape[-2] = nao;
    if (len(shape) == 2): shape.insert(0, 1)
    dx = C @ x.reshape((-1, C.shape[1], nocc)) @ Co.T
    v = np.zeros((int(x.size / C.shape[1] / nocc), nao, nao))
    for i in range(dx.shape[0]):
        dm = dx[i] + dx[i].T
        v[i] = 0.5 * scf_eng.get_j(mol, dm) - 0.25 * c_s_x * scf_eng.get_k(mol, dm)
        v[i] += np.einsum("kluv, kl -> uv", rdm2_inAx, dm)
        v[i] += v[i].T
    return 2 * v.reshape(shape)

def Axpi(x): return  C.T @ Axao(x,  C) @ Co
def Axai(x): return Cv.T @ Axao(x, Cv) @ Co

In [18]:
%%time
def get_cpks_ao_h1(A):
    ao_matrix = np.zeros((3, nao, nao))
    sA = mol_slice(A)
    ao_matrix[:, sA] = (- int1e_ipkin - int1e_ipnuc 
                        - np.einsum("tuvkl, kl -> tuv", int2e_ip1, D)
                        + 0.5 * c_s_x * np.einsum("tukvl, kl -> tuv", int2e_ip1, D)
                       )[:, sA]
    ao_matrix -= np.einsum("tkluv, kl -> tuv", int2e_ip1[:, sA], D[sA])
    ao_matrix += 0.5 * c_s_x * np.einsum("tkulv, kl -> tuv", int2e_ip1[:, sA], D[sA])
    with mol.with_rinv_as_nucleus(A):
        ao_matrix -= mol.intor("int1e_iprinv") * mol.atom_charge(A)
    return ao_matrix + ao_matrix.swapaxes(1, 2)

def get_cpks_ao_s1(A):
    ao_matrix = np.zeros((3, nao, nao))
    sA = mol_slice(A)
    ao_matrix[:, sA] = -int1e_ipovlp[:, sA]
    return ao_matrix + ao_matrix.swapaxes(1, 2)

cpks_ao_h1 = np.array([ get_cpks_ao_h1(A) + cpks_ao_hgga[A] for A in range(natm) ])
cpks_ao_s1 = np.array([ get_cpks_ao_s1(A) for A in range(natm) ])
cpks_pi_h1 = np.einsum("Atuv, up, vi -> Atpi", cpks_ao_h1, C, Co)
cpks_pi_s1 = np.einsum("Atuv, up, vi -> Atpi", cpks_ao_s1, C, Co)

CPU times: user 273 ms, sys: 5.65 ms, total: 278 ms
Wall time: 17.5 ms


In [19]:
%%time
cpks_Upi, _ = scf.cphf.solve(Axpi, e, mo_occ, cpks_pi_h1.reshape(-1, nmo, nocc), cpks_pi_s1.reshape(-1, nmo, nocc))
cpks_Upi.shape = (natm, 3, nmo, nocc)

CPU times: user 2.58 s, sys: 15.5 ms, total: 2.59 s
Wall time: 162 ms


最终我们的任务很简单，即将 $U_{pi}^{A_t}$ 与 $F_{pi}^\mathrm{n}$ 简单地相乘即得到 U 矩阵贡献的部分了：

In [20]:
%%time
grad_Upi = (cpks_Upi * F_n[:, :nocc]).sum(axis=(2, 3)) * 4

CPU times: user 1.19 ms, sys: 205 µs, total: 1.39 ms
Wall time: 102 µs


我们可以验证最终得到的解析梯度与数值梯度基本接近；但如前文所述，可能在解析梯度中没有很好地考虑格点梯度，从而最终的解析梯度与数值梯度的差会稍大：

In [21]:
np.allclose(grad_nuc + grad_skeleton + grad_Upi,
            grad_nscf_numerical, atol=1e-6)

True

### Z-Vector 方程的计算简化

实施上，上面的计算过程可以被简化．上述的过程中，我们对所有的 4 个原子与 3 个坐标分量，即总共 12 个求导变量进行 U 矩阵的求取；我们可以简化为只需要求一个类似于 CP-KS 方程的过程，该方程称为 Z-Vector 方程．

我们回顾 CP-HF 方程

\begin{equation}
(\varepsilon_i - \varepsilon_a) U_{ai}^{A_t} - A_{ai, bj} U_{bj}^{A_t} = B_{ai}^{A_t}
\end{equation}

而我们之所以需要 U 矩阵，是因为我们希望求取项 $4 U_{pi}^{A_t} F_{pi}^\mathrm{n} = - 2 S_{ij}^{A_t} F_{ij}^\mathrm{n} + 4 U_{ai}^{A_t} F_{ai}^\mathrm{n}$；那么我们对等式左边与 $U_{ai}^{A_t}$ 作简单的变换，就得到

\begin{equation}
\big( (\varepsilon_i - \varepsilon_a) \delta_{ai, bj} - A_{ai, bj} \big) U_{bj}^{A_t} F_{bj}^\mathrm{n} (F_{bj}^\mathrm{n})^{-1} = B_{ai}^{A_t}
\end{equation}

我们对等式两边乘以一个新的未确定量 $Z_{ai}$，并且稍对等式左进行处理，则得到下述关系：

\begin{equation}
Z_{ai} \big( (\varepsilon_i - \varepsilon_a) \delta_{ai, bj} - A_{ai, bj} \big) (F_{bj}^\mathrm{n})^{-1} \cdot U_{bj}^{A_t} F_{bj}^\mathrm{n} = Z_{ai} B_{ai}^{A_t}
\end{equation}

如果我们让 $Z_{ai}$ 满足下述关系：

\begin{equation}
Z_{ai} \big( (\varepsilon_i - \varepsilon_a) \delta_{ai, bj} - A_{ai, bj} \big) (F_{bj}^\mathrm{n})^{-1} = 1
\end{equation}

或者等价地：

\begin{equation}
(\varepsilon_i - \varepsilon_a) Z_{ai} - A_{ai, bj} Z_{bj} = F_{ai}^\mathrm{n}
\end{equation}

那么，此时的 $4 Z_{ai} B_{ai}^{A_t} = 4 U_{bj}^{A_t} F_{bj}^\mathrm{n}$ 即是我们希望的 U 矩阵相关的一阶梯度的贡献量了．

而上述方程的形式与 CP-KS 方程完全一致，但可以发现，我们只需要通过一次 CP-KS 过程求一次 $Z_{ai}$ 就能得到我们希望的梯度结果，从而避免计算 12 次耗时的 CP-KS 过程．

通过 CP-KS 过程求取 $Z_{ai}$ 的过程非常简单：

In [22]:
%%time
cpks_Zai, _ = scf.cphf.solve(Axai, e, mo_occ, F_n[nocc:, :nocc])

CPU times: user 142 ms, sys: 1.98 ms, total: 144 ms
Wall time: 9.09 ms


随后我们需要构建 $B_{ai}^{A_t}$ 张量，其构建方式如下：

In [23]:
%%time
cpks_pi_b1 = cpks_pi_h1[:, :, nocc:] - cpks_pi_s1[:, :, nocc:] * eo
cpks_pi_b1 += -0.5 * Cv.T @ Axao(cpks_pi_s1[:, :, :nocc], Co) @ Co

CPU times: user 266 ms, sys: 2.43 ms, total: 269 ms
Wall time: 17.1 ms


那么 $4 Z_{ai} B_{ai}^{A_t}$ 就是一部分 U 矩阵的梯度贡献了：

In [24]:
%%time
grad_Zai = (cpks_Zai * cpks_pi_b1).sum(axis=(2, 3)) * 4

CPU times: user 1.09 ms, sys: 0 ns, total: 1.09 ms
Wall time: 82.3 µs


我们注意到，刚才还从 $4 U_{pi}^{A_t} F_{pi}^\mathrm{n}$ 拆分出 $- 2 S_{ij}^{A_t} F_{ij}^\mathrm{n}$ 一项；该项的计算方式如下：

In [25]:
%%time
grad_SF = - (cpks_pi_s1[:, :, :nocc, :nocc] * F_n[:nocc, :nocc]).sum(axis=(2, 3)) * 2

CPU times: user 1.06 ms, sys: 184 µs, total: 1.25 ms
Wall time: 92 µs


最后，我们可以验证通过 Z-Vector 方程给出的非自洽 GGA 的结果：

In [26]:
np.allclose(grad_nuc + grad_skeleton + grad_Zai + grad_SF,
            grad_nscf_numerical, atol=1e-6)

True